New Attempt

In [347]:
import numpy as np
import random

# Parameters

battery_capacity = 100
battery_level = 0 #this is for the kwh stored which we will convert into a percentage when filling in the q table
actions = [0,1,2]  # Possible actions involing charging and energy drain
time_periods = 48  # 24 hours in the day
max_battery_capacity = 5


battery_C_rate = 2.6   #kwh
interval_length = 24/time_periods   #length in hours
interval_C_rate = battery_C_rate * interval_length

#epsilon = 0.3  # Exploration rate
#alpha = 1  # Learning rate
gamma = 0.99  # Discount factor

epsilon_start = 1.0
epsilon_min = 0.1
alpha_start = 0.5
alpha_min = 0.05
decay_rate = 0.001

# Forecasted data (example values, replace with actual forecasts)
# house_demand = [0.7] * 8 + [0.8] * 4 + [1.2] * 4 + [1] * 4 + [1.5] * 2 + [0.4] * 2 
# solar_forecast = [0] * 8 + [1.6] * 8 + [0] * 8

#30min intervals
house_demand = [0.4/2] * 16 + [0.8/2] * 8 + [1.2/2] * 8 + [0.4/2] * 8 + [2/2] * 4 + [0.3/2] * 4 
solar_forecast = [0] * 16 + [1.2/2] * 16 + [0] * 16

# q-table
q_table = np.zeros((battery_capacity + 1, time_periods, len(actions)))

In [348]:
def get_reward(time, battery_level, action, solar_forecast, house_demand, excess_charge):
    """
    Returns the reward, which is then used to update the value for this state-action pair.

    Parameters:
        time (int): An integer between 0 and 23 representing the current hour.
        battery_level (int): Integer representing current battery charge between 0 and 100.
        action (int): Integer between 0-2 representing which action to take.
        solar_forecast (array): Forecasted solar energy for each hour of the day in kWh.
        house_demand (array): Forecasted power requirement of the house for each hour in kWh.

    Returns:
        float: Calculated reward for the given action and state.
        
    """
  
    # Define grid costs based on time of day
    if 2 <= time < 5:  # Cheap charging hours (2-5 AM)
        grid_cost = 0.02
    elif 16 <= time < 19:  # Expensive hours (4-7 PM)
        grid_cost = 0.32
    else:
        grid_cost = 0.23 # Normal hours

    # Get the power needed and solar availability for this specific hour 
    solar_available = solar_forecast[time]
    house_used = house_demand[time]

    # Calculate energy flows for the hour
    solar_used = min(solar_available, house_used)
    hourly_demand = house_used - solar_used

    
    #if when updating the battery there has been excess sun set a small penalty for the wasted sun
    reward = -excess_charge * 0.02

    # Reward Calculation
    if action == 0:  # Charge battery, power house from grid
        reward += -grid_cost * (hourly_demand + interval_C_rate)
    elif action == 1:  # No battery charge, power house from grid
        reward += -grid_cost * hourly_demand
    elif action == 2:  # No battery charge, power house from battery + grid if needed
    
        if hourly_demand >= interval_C_rate:  # hourly demand is higher than battery can provide
            reward -= (min(interval_C_rate, battery_level)) #* 0.01 + (hourly_demand - min(interval_C_rate, battery_level)) * grid_cost  # Small cost for using battery
        else:  # demand is low so potentially battey fully covers it 
        
            grid_needed = max((hourly_demand - min(interval_C_rate, battery_level)), 0)
            reward -= (grid_cost * grid_needed) #+ min(hourly_demand, battery_level ) * 0.01
            
        #penalise if not drained at the end
    if  time == 23 and battery_level > 0 :
         reward -= 2
    
    return reward


In [349]:
total_reward = 0

test_reward = get_reward(22, 0.2, 2 ,solar_forecast, house_demand, excess_charge = 0)
print(test_reward)
total_reward += test_reward
# test_reward = get_reward(23, 0.2, 1 ,solar_forecast, house_demand, excess_charge = 0)
# print(test_reward)
# total_reward += test_reward

#print(total_reward)



0.0


In [350]:
def get_cost(time, battery_level, action, solar_forecast, house_demand, excess_charge):
    """
    Returns the cost of what the real life system should cost us for the energy used

    Parameters:
        time (int): An integer between 0 and 23 representing the current hour.
        battery_level (int): Integer representing current battery charge between 0 and 100.
        action (int): Integer between 0-2 representing which action to take.
        solar_forecast (array): Forecasted solar energy for each hour of the day in kWh.
        house_demand (array): Forecasted power requirement of the house for each hour in kWh.

    Returns:
        float: Calculated reward for the given action and state.
        
    """
  
    # Define grid costs based on time of day
    if 2 <= time < 5:  # Cheap charging hours (2-5 AM)
        grid_cost = 0.14
    elif 16 <= time < 19:  # Expensive hours (4-7 PM)
        grid_cost = 0.32
    else:
        grid_cost = 0.23  # Normal hours

    # Get the power needed and solar availability for this specific hour 
    solar_available = solar_forecast[time]
    house_used = house_demand[time]

    # Calculate energy flows for the hour
    solar_used = min(solar_available, house_used)
    hourly_demand = house_used - solar_used

    
    #if when updating the battery there has been excess sun set a small penalty for the wasted sun
    #reward = -excess_charge * 0.01
    reward = 0
    # Reward Calculation
    if action == 0:  # Charge battery, power house from grid
        reward += grid_cost * (hourly_demand + interval_C_rate)
    elif action == 1:  # No battery charge, power house from grid
        reward += grid_cost * hourly_demand
    elif action == 2:  # No battery charge, power house from battery + grid if needed
    
        if hourly_demand >= interval_C_rate:  # hourly demand is higher than battery can provide
            reward +=  (hourly_demand - min(interval_C_rate, battery_level)) * grid_cost  # Small cost for using battery
        else:  # demand is low so potentially battey fully covers it 
            grid_needed = max((hourly_demand - min(interval_C_rate, battery_level)), 0)
            reward += (grid_cost * grid_needed)

    # Penalize if battery gets too low before peak hour
    
    return reward

In [351]:
#Q - Learning part
for episode in range(100000):  # Number of episodes

    current_battery_level = 0  # Start with an empty battery at midnight
    current_battery_capacity = 0

    for time in range(time_periods): # loop over each hour

        useful_charge = 0
        excess_charge = 0

        epsilon = epsilon_min + (epsilon_start - epsilon_min) * np.exp(-decay_rate * episode)
        alpha = alpha_min + (alpha_start - alpha_min) * np.exp(-decay_rate * episode)

        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)  # explore
        else:
            action = np.argmax(q_table[current_battery_capacity, time])  # use current max
        

        # Get solar availability and house demand for this hour
        solar_available = solar_forecast[time]
        house_used = house_demand[time]

        #first step is to alter the battery level based on the sun levels
        if solar_available > house_used:
            potential_charge = solar_available - house_used
        
            # what can actually be used 
            useful_charge = min(potential_charge, max_battery_capacity - current_battery_level)
            #print(f'useful charge: {useful_charge}')

            excess_charge = potential_charge - useful_charge
            #print(f'excess charge: {excess_charge}')



        predicted_battery_level = current_battery_level + useful_charge
        #print(f'actoin chosen = {action}  predicted_battery_level after actions = {predicted_battery_level} and time = {time}')


        #new_battery_capacity = int(round((predicted_battery_level / 5) * 100))

        # Update battery level based on action
        if action == 0:  # Charge and power house from grid
             
            predicted_battery_level = min(max_battery_capacity, predicted_battery_level + interval_C_rate)

        elif action == 1:  # No battery charge, power house from grid
            pass

        elif action == 2:  # No battery charge and power house from battery if possible
            # here if solar is higher than house used then nothing will be changed 
            # if solar is less then we change the battery level to reflect we have powered the house via the battery
            if solar_available < house_used:
                interval_drain =  house_used - solar_available
                # caps the drain at 2.6 and makes sure battery stays above 0
                
                if interval_drain >= interval_C_rate:
                    predicted_battery_level = max(0, predicted_battery_level - interval_C_rate) 
                else:    
                    predicted_battery_level = max(0, predicted_battery_level - interval_drain) 



        #print(f'actoin chosen = {action}  predicted_battery_level after actions = {predicted_battery_level} and time = {time}')
        #print(f'actoin chosen = {action}  current_battery_level after actions = {current_battery_level} and time = {time}')
            
           

        #get the reward for this state and action
        reward = get_reward(time, current_battery_level, action, solar_forecast, house_demand, excess_charge)
        #print(f'actoin chosen = {action}  reward with predicted = {reward} and time = {time}')
        #reward = get_reward(time, current_battery_level, action, solar_forecast, house_demand, excess_charge)
        #print(f'actoin chosen = {action}  reward with current = {reward} and time = {time}')
        
        # gets the capacity for the state in the table this is after we have performed the action so is the next state
        new_battery_capacity = int(round((predicted_battery_level / max_battery_capacity) * 100))



        # next state is also the next time period
        next_time = (time + 1) % time_periods
        #get max value
        next_max = np.max(q_table[new_battery_capacity, next_time])

        # get the current states q-table value
        old_value = q_table[current_battery_capacity, time, action]

        #update it using the action reward, next state max and old value  (bellman equation)
        q_table[current_battery_capacity, time, action] = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)


        #change to the new state,

        current_battery_capacity = new_battery_capacity
        current_battery_level = predicted_battery_level 
        time = next_time

        


In [352]:

print("Trained Q-table sample for (Battery Level, Time):")
for battery_level in range(0, 101, 2):
    for time in [ 21,22]:  # Sample times across the day
        print(f"Battery Level: {battery_level}, Time: {time}, Q-values: {q_table[battery_level, time]}")


Trained Q-table sample for (Battery Level, Time):
Battery Level: 0, Time: 21, Q-values: [0. 0. 0.]
Battery Level: 0, Time: 22, Q-values: [0. 0. 0.]
Battery Level: 2, Time: 21, Q-values: [0. 0. 0.]
Battery Level: 2, Time: 22, Q-values: [0. 0. 0.]
Battery Level: 4, Time: 21, Q-values: [0. 0. 0.]
Battery Level: 4, Time: 22, Q-values: [0. 0. 0.]
Battery Level: 6, Time: 21, Q-values: [0. 0. 0.]
Battery Level: 6, Time: 22, Q-values: [0. 0. 0.]
Battery Level: 8, Time: 21, Q-values: [0. 0. 0.]
Battery Level: 8, Time: 22, Q-values: [0. 0. 0.]
Battery Level: 10, Time: 21, Q-values: [0. 0. 0.]
Battery Level: 10, Time: 22, Q-values: [0. 0. 0.]
Battery Level: 12, Time: 21, Q-values: [0. 0. 0.]
Battery Level: 12, Time: 22, Q-values: [0. 0. 0.]
Battery Level: 14, Time: 21, Q-values: [0. 0. 0.]
Battery Level: 14, Time: 22, Q-values: [0. 0. 0.]
Battery Level: 16, Time: 21, Q-values: [0. 0. 0.]
Battery Level: 16, Time: 22, Q-values: [0. 0. 0.]
Battery Level: 18, Time: 21, Q-values: [0. 0. 0.]
Battery Le

In [353]:
def get_optimal_action_sequence(q_table, current_battery_level, time_periods):
    #initialise starting state
    timestep_data = []
    cost = []
    cumulative_cost = 0

    for time in range(time_periods):

        battery_capacity = int(round((current_battery_level / max_battery_capacity) * 100))


        # Find the action with the maximum Q-value for the current state
        action = np.argmax(q_table[battery_capacity, time])
        #print(f'battery_capacity after actions = {battery_capacity} and time = {time}')

        #store the data
        timestep_data.append({
                'time': time,
                'battery_capacity': battery_capacity,
                'action': action
            })


        useful_charge = 0
        #excess not needed as we arent doing any reward stuff
        #excess_charge = 0

        # Get solar availability and house demand for this hour
        solar_available = solar_forecast[time]
        house_used = house_demand[time]

        #first step is to alter the battery level based on the sun levels
        if solar_available > house_used:
            potential_charge = solar_available - house_used
        
            # what can actually be used 
            useful_charge = min(potential_charge, max_battery_capacity - current_battery_level)

            #excess_charge = potential_charge - useful_charge

        predicted_battery_level = current_battery_level + useful_charge


        # Update battery level based on action
        if action == 0:  # Charge and power house from grid
             
            predicted_battery_level = min(max_battery_capacity, predicted_battery_level + interval_C_rate)

        elif action == 1:  # No battery charge, power house from grid
            pass

        elif action == 2:  # No battery charge and power house from battery if possible
            # here if solar is higher than house used then nothing will be changed 
            # if solar is less then we change the battery level to reflect we have powered the house via the battery
            if solar_available < house_used:
                interval_drain =  house_used - solar_available
                # caps the drain at 2.6 and makes sure battery stays above 0
                if interval_drain >= interval_C_rate:
                    predicted_battery_level = max(0, predicted_battery_level - interval_C_rate) 
                else:    
                    predicted_battery_level = max(0, predicted_battery_level - interval_drain) 


        #get reward for the action and state
        cost.append(get_cost(time, current_battery_level, action, solar_forecast, house_demand, excess_charge))
        cumulative_cost += get_cost(time, current_battery_level, action, solar_forecast, house_demand, excess_charge)



        # gets the capacity for the state in the table this is after we have performed the action so is the next state
        
        current_battery_level = predicted_battery_level

    return timestep_data, cumulative_cost,cost

# Get optimal actions for a day starting from initial battery level
optimal_actions = get_optimal_action_sequence(q_table, 0, time_periods)



In [354]:
type(optimal_actions)

tuple

In [355]:
# Print list
actions , reward,step_cost = optimal_actions
print(reward)   

for element in actions:
    print(element)

1.4005
{'time': 0, 'battery_capacity': 0, 'action': 1}
{'time': 1, 'battery_capacity': 0, 'action': 1}
{'time': 2, 'battery_capacity': 0, 'action': 0}
{'time': 3, 'battery_capacity': 26, 'action': 0}
{'time': 4, 'battery_capacity': 52, 'action': 0}
{'time': 5, 'battery_capacity': 78, 'action': 2}
{'time': 6, 'battery_capacity': 74, 'action': 2}
{'time': 7, 'battery_capacity': 70, 'action': 2}
{'time': 8, 'battery_capacity': 66, 'action': 2}
{'time': 9, 'battery_capacity': 62, 'action': 2}
{'time': 10, 'battery_capacity': 58, 'action': 2}
{'time': 11, 'battery_capacity': 54, 'action': 2}
{'time': 12, 'battery_capacity': 50, 'action': 2}
{'time': 13, 'battery_capacity': 46, 'action': 2}
{'time': 14, 'battery_capacity': 42, 'action': 2}
{'time': 15, 'battery_capacity': 38, 'action': 2}
{'time': 16, 'battery_capacity': 34, 'action': 1}
{'time': 17, 'battery_capacity': 38, 'action': 1}
{'time': 18, 'battery_capacity': 42, 'action': 1}
{'time': 19, 'battery_capacity': 46, 'action': 1}
{'time

In [356]:
step_cost

[0.046000000000000006,
 0.046000000000000006,
 0.21000000000000002,
 0.21000000000000002,
 0.21000000000000002,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.046000000000000006,
 0.0,
 0.529,
 0.0,
 0.0,
 0.0,
 0.0345,
 0.0345,
 0.0345]